## Example curve prediction script
In this notebook we will try to predict the function coefficients.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


In [ ]:
# function to predict
def func(x, a, b, c):
    # 2nd degree polynomial
    # return a * x**2 + b * x + c

    # random, non-linear function
    return a * x**2 * np.sin(b * x) + c * x

In [ ]:
# Load the data
df = pd.read_pickle('./data/random-curve.pkl')
df.head()

In [ ]:
# The targets are m, s
param_a = df.pop('a')
param_b = df.pop('b')
param_c = df.pop('c')
# All x should be equal
x = df.iloc[0].x
df.pop('x')

In [ ]:
print(x)

In [ ]:
df.head()

In [ ]:
values = np.zeros((len(df), len(df.iloc[0,0])), dtype=float)
# print(values.shape)
for index, row in df.iterrows():
    values[index, :] = row.values[0]
print(values.shape)
print(type(values))

In [ ]:
# Now I need to start setting up the NN with keras
tf_data = tf.convert_to_tensor(values)
print(tf_data)

In [ ]:
# tf_target = tf.convert_to_tensor(np.array([param_a, param_b, param_c]))
target = np.zeros(shape=(len(param_a), 3), dtype=float)
target[:, 0] = np.array(param_a)
target[:, 1] = np.array(param_b)
target[:, 2] = np.array(param_c)
tf_target = tf.convert_to_tensor(target)
print(tf_target)

In [ ]:
# Define a simple model

def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(64, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

In [ ]:
# evaluate our model
def evaluate_model(X, y):
    results = list()
    n_inputs = X.shape[1]
    n_outputs = y.shape[1]
        # define evaluation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = tf.gather(X, train_ix), tf.gather(X, test_ix)
        y_train, y_test = tf.gather(y, train_ix), tf.gather(y, test_ix)

        # X_train, X_test = tf_data[train_ix], tf_data[test_ix]
        # y_train, y_test = tf_target[train_ix], tf_target[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mae)
        results.append(mae)
    print('MAE: %3f (%.3f)' % (np.mean(results), np.std(results)))
    return results

In [ ]:
n_inputs = tf_data.shape[1]
n_outputs = tf_target.shape[1]
train_ix, test_ix = train_test_split(np.arange(tf_data.shape[0]), test_size=0.05)
X_train, X_test = tf.gather(tf_data, train_ix), tf.gather(tf_data, test_ix)
y_train, y_test = tf.gather(tf_target, train_ix), tf.gather(tf_target, test_ix)
model = get_model(n_inputs, n_outputs)
# fit model
print('Fitting the model')
history = model.fit(X_train, y_train, validation_split=0.05, verbose=0, epochs=400)
# evaluate model on test set
print('Evaluating the model')
mae = model.evaluate(X_test, y_test, verbose=1)
# store result
print('MAE: %.3f' % mae)

In [ ]:
# plot history
%matplotlib widget

def plot_loss(loss, val_loss, epoch=(-1, -1)):
    fig = plt.figure()

    if epoch[0] == -1:
        start_idx = 0
    else:
        start_idx = epoch[0]

    if epoch[1] == -1:
        end_idx = len(loss)
    else:
        end_idx = epoch[1]
    loss = loss[start_idx: end_idx]
    val_loss = val_loss[start_idx: end_idx]
    plt.plot(np.arange(start_idx, end_idx), loss, label='train')
    plt.plot(np.arange(start_idx, end_idx), val_loss, label='validation')
    plt.title('Model Accuracy')
    plt.ylabel('MAE')
    plt.xlabel('epoch')
    plt.legend()
    plt.show()

plot_loss(history.history['loss'], history.history['val_loss'], epoch=(-1,-1))

In [ ]:
# make predictions, and plot them
random_idx = np.random.choice(np.arange(len(X_test)), size=5, replace=False)
X_validate = tf.gather(X_test, random_idx)
y_validate = tf.gather(y_test, random_idx)
yhat = model.predict(X_validate)
print('Original: ', y_validate)
print('Predicted:', yhat)

In [ ]:
# plot the validation curves
# %matplotlib inline
%matplotlib widget
from scipy.optimize import curve_fit
plt.figure(figsize=(10, 10))
for idx in range(len(y_validate)):
    y_real = func(x, *y_validate[idx])
    y_pred = func(x, *yhat[idx])
    popt, pcov  = curve_fit(func, x, X_validate[idx])
    p = plt.plot(x, y_real, label=f'y_real{idx}', ls='-')
    plt.plot(x, y_pred, label=f'y_pred{idx}', ls='--', color=p[0].get_color())
    plt.plot(x, func(x, *popt), label=f'y_opt{idx}', ls=':', color=p[0].get_color())
plt.legend(ncol=3)
plt.show()